In [1]:
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn import preprocessing
from scipy.stats import norm
import pomegranate

# Getting data and diving them into unique unit numbers

We need to divide data into unique numbers, because the state restes as the unit number changes, so we need to find Gaussian distribution for different unit numbers

In [2]:
data = pd.read_csv('~/Documents/hitachi/CMAPSS/train_FD001.txt', sep=" ", header=None)
unique_unit_values = data[0].unique() #Number of units
data_cycles = []
for unit_num in unique_unit_values:
    data_cycles.append(data[data[0] == unit_num])

# Removing operational settings and normalize the data column wise

In [3]:
def normalize(data):
    x = data.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    dataNew = pd.DataFrame(x_scaled)
    return dataNew
#Remove the operation settings
dataT = data[data.columns[5:26]]
dataT.columns = range(21)
dataT = normalize(dataT)

# Dividing data for each unit

I think this is why my transitional matrix previously was not working properly as in each unit the state resets and start from good condition

In [4]:
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

# Identifying and removing non variable data columns

Removing the columns where the data does not vary

In [5]:
for dataT_cycle in dataT_cycles:
    print(dataT_cycle.columns[dataT_cycle.std() == 0])
"""
Here we can see 0,4,9,15,17,18 but also 5 at many places so we drop column number 5 as well
"""
dataT.drop(data.columns[[0, 3, 4, 5, 9, 15, 17, 18]],axis=1,inplace=True)
dataT.columns = range(13)
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='i

In [6]:
# Right now only using the first data frame (i.e Machine 1) to train the VAE, but we can combine all the dataframes
# and train the VAE jointly on the entire data for better performance 

x_train = dataT_cycles[0].values[:150]
x_test = dataT_cycles[0].values[151:198]
x_train.shape
# x_test.shape

(150, 13)

# Variational AutoEncoders to find Latent State Space Distribution



In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

ModuleNotFoundError: No module named 'keras'

In [8]:
# Data preparation
x_train = dataT_cycles[0].values[:100]
x_test = dataT_cycles[0].values[101:198]
x_train.shape
x_test.shape
original_dim = x_train[0].shape[0]

In [9]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 9
batch_size = 10
latent_dim = 5
epochs = 50

In [10]:
# Sampling function
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


In [11]:
# VAE Model Encoder + Decoder 

# Building the Encoder
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 13)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 9)            126         encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 5)            50          dense_1[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 5)            50          dense_1[0][0]                    
____________________________________________________________________________________________

In [12]:
#build decoder model 
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 54        
_________________________________________________________________
dense_3 (Dense)              (None, 13)                130       
Total params: 184
Trainable params: 184
Non-trainable params: 0
_________________________________________________________________


In [13]:
#instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

In [14]:
def main(args):
    parser = argparse.ArgumentParser()
    help_ = "Load h5 model trained weights"
    parser.add_argument("-w", "--weights", help=help_)
    help_ = "Use mse loss instead of binary cross entropy (default)"
    parser.add_argument("-m", "--mse", help=help_, action='store_true')
    
    models = (encoder, decoder)
    data = (x_test, None)
    
    # VAE loss = mse_loss or xent_loss + kl_loss
    if args.mse:
        reconstruction_loss = mse(inputs, outputs)
    else:
        reconstruction_loss = binary_crossentropy(inputs, outputs)
        
    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis= -1)
    kl_loss *= 0.5 
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    vae.summary()
    
    if args.weights:
        vae.load_weights(args.weights)
    else:
        # Train the autoencoder
        vae.fit(x_train, epochs=epochs, batch_size= batch_size, validation_data=(x_test, None))
        vae.save_weights('vae_mlp_CMAPSS.h5')

In [15]:
class Args:
    mse = None
    weights = None
    
args = Args()

if __name__ == '__main__':
    main(args)
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "vae_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 13)                0         
_________________________________________________________________
encoder (Model)              [(None, 5), (None, 5), (N 226       
_________________________________________________________________
decoder (Model)              (None, 13)                184       
Total params: 410
Trainable params: 410
Non-trainable params: 0
_________________________________________________________________


/home/yannik/miniconda3/envs/ykp/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))



Train on 100 samples, validate on 91 samples
Epoch 1/50
100/100 [==============================] - 1s 6ms/step - loss: 8.6136 - val_loss: 7.5069
Epoch 2/50
100/100 [==============================] - 0s 159us/step - loss: 8.1071 - val_loss: 6.7097
Epoch 3/50
100/100 [==============================] - 0s 176us/step - loss: 7.4209 - val_loss: 5.7314
Epoch 4/50
100/100 [==============================] - 0s 188us/step - loss: 6.5338 - val_loss: 4.2211
Epoch 5/50
100/100 [==============================] - 0s 188us/step - loss: 5.2482 - val_loss: 2.3457
Epoch 6/50
100/100 [==============================] - 0s 222us/step - loss: 3.4696 - val_loss: -0.6214
Epoch 7/50
100/100 [==============================] - 0s 234us/step - loss: 0.8200 - val_loss: -4.6524
Epoch 8/50
100/100 [==============================] - 0s 233us/step - loss: -3.1063 - val_loss: -10.8377
Epoch 9/50
100/100 [==============================] - 0s 199us/step - loss: -8.8391 - val_loss: -20.2851
Epoch 10/50
100/100 [=========

In [16]:
# Once the VAE has been trained, we can use the encoder to sample the latent space

#predicting the latent space for first 13 observation values for machine 1
test = np.asarray(x_train[0:13])  

# indexing on 2 because the encoder predicts z_mean, z_log_var and sampled vector z (we are interested in z only)
latent_space = encoder.predict(test)[2]  

In [17]:
# Each list is a 5 dimension latent state space for that observation value
latent_space

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]], dtype=float32)

In [18]:
# Reconstruct the raw observation from the learned latent space 
sample = decoder.predict(latent_space)

In [19]:
# Compare the with the real x_train value
x_train[0]

array([0.18373494, 0.40680183, 0.72624799, 0.24242424, 0.109755  ,
       0.36904762, 0.63326226, 0.20588235, 0.1996078 , 0.36398615,
       0.33333333, 0.71317829, 0.7246617 ])

In [20]:
# Right now they are not same as we trained the VAE on very less amount of data 
sample[0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype=float32)

# Using HMM to find out transitional matrices

Here we first define transmatrix as [[0.5, 0.5, 0.0, 0.0],[0.0, 0.4, 0.6, 0.0],[0.0, 0.0, 0.3, 0.7],[0.0,0.0,0.0,1.0]] which means there is half chance for each state to go to next state and half to remain in the current state itself when fully healthy, 0.4 probability of remaining at current state when at 'above average' health, 0.3 probability when 'below average', and with certainty to stay at a failing state if already failing.

Then we train for each unit for transmatrix as well as state means and we will take average of each unit transmatrices and states as the transmatrix and state

*Note*: Here state '0' means the perfect health and '3' means weakest health 

In [7]:
dataT_cycles[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.183735,0.406802,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662
1,0.283133,0.453019,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014
2,0.343373,0.369523,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375
3,0.343373,0.256159,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386
4,0.349398,0.257467,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.765060,0.683235,0.336554,0.621212,0.072602,0.684524,0.234542,0.514706,0.091599,0.753367,0.666667,0.286822,0.089202
188,0.894578,0.547853,0.136876,0.560606,0.102396,0.732143,0.189765,0.661765,0.090670,0.744132,0.583333,0.263566,0.301712
189,0.731928,0.614345,0.231884,0.590909,0.084582,0.880952,0.287846,0.691176,0.065229,0.759523,0.833333,0.271318,0.239299
190,0.641566,0.682799,0.172303,0.575758,0.094364,0.773810,0.187633,0.617647,0.075704,0.740669,0.500000,0.240310,0.324910


In [8]:
lr = hmm.GaussianHMM(n_components=4, covariance_type="diag",init_params="cm", params="mtc")
lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
transmats = []
statemeans = []
covars = []
for i in range(100):
    lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.4, 0.6, 0.0],[0.0, 0.0, 0.3, 0.7],[0.0,0.0,0.0,1.0]])
    lr.fit(dataT_cycles[i])
    transmats.append(lr.transmat_)
    statemeans.append(lr.means_)
    covars.append(lr.covars_)

In [9]:
#lr = hmm.GMMHMM(n_components=4, n_mix=4, covariance_type="diag",init_params="cm", params="mt")
#lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
#transmats = []
#statemeans = []
#for i in range(100):
#    lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]])
#    lr.fit(dataT_cycles[i])
#    transmat = lr.transmat_
#    transmats.append(transmat)
#    statemeans.append(lr.means_)

In [10]:
transmat = np.array(transmats).mean(axis=0)
statemean = np.array(statemeans).mean(axis=0)
covar = np.array(covars).mean(axis=0)

In [11]:
transmat

array([[0.59505558, 0.40494442, 0.        , 0.        ],
       [0.        , 0.76482211, 0.23517789, 0.        ],
       [0.        , 0.        , 0.88093494, 0.11906506],
       [0.        , 0.        , 0.        , 0.98      ]])

In [12]:
pd.DataFrame(transmat)

,0,1,2,3
0,0.595056,0.404944,0.000000,0.000000
1,0.000000,0.764822,0.235178,0.000000
2,0.000000,0.000000,0.880935,0.119065
3,0.000000,0.000000,0.000000,0.980000


In [13]:
pd.DataFrame(statemean)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.361872,0.358909,0.649480,0.250338,0.159575,0.319248,0.673117,0.270409,0.197400,0.383784,0.363723,0.599998,0.627357
1,0.447684,0.432294,0.555328,0.302983,0.188051,0.422327,0.574544,0.327012,0.216596,0.462189,0.440688,0.513007,0.535993
2,0.566330,0.537478,0.434005,0.378785,0.241778,0.567415,0.431069,0.399449,0.259748,0.582052,0.541015,0.402448,0.411874
3,0.700865,0.638011,0.308127,0.475684,0.310537,0.720825,0.284764,0.494123,0.319494,0.704721,0.656289,0.275199,0.283998


In [50]:
t_prob = np.array([transmat, transmat])

In [51]:
rewards = np.array([[100, 50, 0, -50],[-50, 0, 50, 100]])

In [52]:
pd.DataFrame(rewards)

,0,1,2,3
0,100,50,0,-50
1,-50,0,50,100


In [53]:
pd.DataFrame(e_prob[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.374788,0.376223,0.324514,0.387588,0.394292,0.381208,0.320688,0.385311,0.391740,0.372861,0.374594,0.335262,0.329673
1,0.364689,0.368252,0.336731,0.383089,0.392593,0.370781,0.335680,0.381324,0.390411,0.363984,0.365500,0.346336,0.341720
2,0.350527,0.354387,0.354969,0.375980,0.388699,0.352671,0.354949,0.373650,0.386448,0.347917,0.352987,0.361741,0.359971
3,0.322227,0.331848,0.377370,0.361736,0.381070,0.318619,0.379061,0.358460,0.379796,0.318186,0.329709,0.381543,0.380118


In [54]:
covar

array([[[0.00938058, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.00884829, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.00721507, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.00590842, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.00490603,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.

In [55]:
covar=[covar[i].sum(axis=1) for i in range(4)]

In [43]:
pd.DataFrame(covar)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.009381,0.008848,0.007215,0.005908,0.004906,0.007176,0.007268,0.005874,0.004842,0.008164,0.008276,0.008201,0.008689
1,0.009564,0.009028,0.006730,0.004793,0.003711,0.006666,0.006836,0.004802,0.003609,0.007926,0.008033,0.007727,0.008710
2,0.009819,0.008933,0.006580,0.004251,0.003441,0.006691,0.007031,0.004233,0.003233,0.007880,0.008255,0.007947,0.008785
3,0.010926,0.009733,0.007825,0.005207,0.003204,0.008772,0.008690,0.005037,0.003014,0.009121,0.008772,0.009382,0.010199


In [56]:
e_prob = np.array([norm.pdf(statemean,covar), norm.pdf(statemean,covar)])

In [57]:
e_prob

array([[[0.37478796, 0.37622323, 0.32451365, 0.38758829, 0.39429238,
         0.38120786, 0.32068759, 0.3853114 , 0.39173988, 0.37286105,
         0.37459408, 0.33526193, 0.32967326],
        [0.36468882, 0.36825229, 0.3367309 , 0.38308862, 0.39259275,
         0.37078129, 0.33568006, 0.3813236 , 0.39041098, 0.36398436,
         0.36549971, 0.34633639, 0.34171987],
        [0.35052656, 0.35438711, 0.35496939, 0.3759799 , 0.388699  ,
         0.35267102, 0.35494918, 0.37365004, 0.38644814, 0.34791691,
         0.35298654, 0.36174134, 0.35997111],
        [0.3222267 , 0.33184765, 0.37737027, 0.36173636, 0.3810701 ,
         0.31861851, 0.37906054, 0.35845975, 0.37979588, 0.31818635,
         0.32970924, 0.38154338, 0.38011795]],

       [[0.37478796, 0.37622323, 0.32451365, 0.38758829, 0.39429238,
         0.38120786, 0.32068759, 0.3853114 , 0.39173988, 0.37286105,
         0.37459408, 0.33526193, 0.32967326],
        [0.36468882, 0.36825229, 0.3367309 , 0.38308862, 0.39259275,
         

In [23]:
# 0: no-repair, 1: repair
actions = ('0', '1')
# 0: failing, 1: low health, 2: good health, 3: perfect health
states = ('0', '1', '2', '3')

discount = 0.95

In [24]:
"""
First we define an MDP. We also represent a policy
as a dictionary of {state: action} pairs, and a utility function as a
dictionary of {state: number} pairs. We then define the value_iteration
and policy_iteration algorithms."""


import random
import numpy as np
from collections import defaultdict

class MDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state."""

    def __init__(self, init, actlist, terminals, transitions=None, reward=None, states=None, discount=0.9):
        if not (0 < discount <= 1):
            raise ValueError("An MDP must have 0 < discount <= 1")

        # collect states from transitions table if not passed.
        self.states = states or self.get_states_from_transitions(transitions)
            
        self.init = init
        
        if isinstance(actlist, list):
            # if actlist is a list, all states have the same actions
            self.actlist = actlist

        elif isinstance(actlist, dict):
            # if actlist is a dict, different actions for each state
            self.actlist = actlist
        
        self.terminals = terminals
        self.transitions = transitions or {}
        if not self.transitions:
            print("Warning: Transition table is empty.")
            
        self.discount = discount
        
        # maybe I should change this
        # self.gamma = gamma

        self.reward = reward or {s: 0 for s in self.states}

        # self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""

        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""

        if not self.transitions:
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set(tr[1] for actions in transitions.values()
                     for effects in actions.values()
                     for tr in effects)
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):

        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)

        # check that init is a valid state
        assert self.init in self.states

        # check reward for each state
        assert set(self.reward.keys()) == set(self.states)

        # check that all terminals are valid states
        assert all(t in self.states for t in self.terminals)

        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001

class POMDP(MDP):

    """A Partially Observable Markov Decision Process, defined by
    a transition model P(s'|s,a), actions A(s), a reward function R(s),
    and a sensor model P(e|s). We also keep track of a gamma value,
    for use by algorithms. The transition and the sensor models
    are defined as matrices. We also keep track of the possible states
    and actions for each state."""

    def __init__(self, actions, transitions=None, evidences=None, rewards=None, states=None, discount=0.95):
        """Initialize variables of the pomdp"""

        if not (0 < discount <= 1):
            raise ValueError('A POMDP must have 0 < discount <= 1')

        self.states = states
        self.actions = actions

        # transition model cannot be undefined
        self.t_prob = transitions
        if not self.t_prob.any():
            print('Warning: Transition model is undefined')
        
        # sensor model cannot be undefined
        self.e_prob = evidences
        if not self.e_prob.any():
            print('Warning: Sensor model is undefined')
        
        self.discount = discount
        # may have to change this
        # self.gamma = gamma
        self.rewards = rewards
        
class Matrix:
    """Matrix operations class"""

    @staticmethod
    def add(A, B):
        """Add two matrices A and B"""

        res = []
        for i in range(len(A)):
            row = []
            for j in range(len(A[0])):
                row.append(A[i][j] + B[i][j])
            res.append(row)
        return res

    @staticmethod
    def scalar_multiply(a, B):
        """Multiply scalar a to matrix B"""

        for i in range(len(B)):
            for j in range(len(B[0])):
                B[i][j] = a * B[i][j]
        return B

    @staticmethod
    def multiply(A, B):
        """Multiply two matrices A and B element-wise"""

        matrix = []
        for i in range(len(B)):
            row = []
            for j in range(len(B[0])):
                row.append(B[i][j] * A[j][i])
            matrix.append(row)

        return matrix

    @staticmethod
    def matmul(A, B):
        """Inner-product of two matrices"""

        return [[sum(ele_a*ele_b for ele_a, ele_b in zip(row_a, col_b)) for col_b in list(zip(*B))] for row_a in A]

    @staticmethod
    def transpose(A):
        """Transpose a matrix"""
        
        return [list(i) for i in zip(*A)]


## Solving POMDP

In [25]:
from scipy.optimize import linprog
import numpy as np
from itertools import product

In [26]:
class AlphaVector:
    """
    Simple wrapper for an alpha vector, used for representing the value function for a POMDP as a piecewise-linear,
    convex function
    """
    def __init__(self, a, v):
        self.action = a
        self.v = v

    def copy(self):
        return AlphaVector(self.action, self.v)

In [27]:
class ValueIteration:
    def __init__(self, pomdp):
        """
        Initialize the POMDP exact value iteration solver
        :param agent:
        :return:
        """
        self.model = pomdp
        self.gamma = set()

    def value_iteration(self, horizon):
        """
        Solve the POMDP by computing all alpha vectors
        :param t: transition probability matrix
        :param o: observation probability matrix
        :param r: immediate rewards matrix
        :param horizon: integer valued scalar represented the number of planning steps
        :return:
        """
        t = self.model.t_prob
        o = self.model.e_prob
        r = self.model.reward
        # the above three are used later--important
        discount = self.model.discount
        actions = len(self.model.actions)  # |A| actions
        states = len(self.model.states)  # |S| states
        # I might need to change this
        observations = len(self.model.get_all_observations())  # |Z| observations
        first = True

        # initialize gamma with a 0 alpha-vector
        dummy = AlphaVector(a=-1, v=np.zeros(states))
        self.gamma.add(dummy)

        # start with 1 step planning horizon, up to horizon-length planning horizon
        for k in range(horizon):
            print('[Value Iteration] planning horizon {}...'.format(k))
            # new set of alpha vectors to add to set gamma
            gamma_k = set()
            # Compute the new coefficients for the new alpha-vectors
            v_new = np.zeros(shape=(len(self.gamma), actions, observations, states))
            idx = 0
            for v in self.gamma:
                for u in range(actions):
                    for z in range(observations):
                        for j in range(states):
                            for i in range(states):
                                # v_i_k * p(z | x_i, u) * p(x_i | u, x_j)
                                v_new[idx][u][z][i] += v.v[i] * o[u][i][z] * t[u][j][i]
                idx += 1
            # add (|A| * |V|^|Z|) alpha-vectors to gamma, |V| is |gamma_k|
            for u in range(actions):
                c = self.compute_indices(idx, observations)
                for indices in c:  # n elements in c is |V|^|Z|
                    for z in range(observations):
                        temp = np.zeros(states)
                        for i in range(states):
                            temp[i] = discount * (r[u][i] + v_new[indices[z]][u][z][i])
                        gamma_k.add(AlphaVector(a=u, v=temp))
            self.gamma.update(gamma_k)
            if first:
                # remove the dummy alpha vector
                self.gamma.remove(dummy)
                first = False
            self.prune(states)
            #  plot_gamma(title='V(b) for horizon T = ' + str(k + 1), self.gamma)

    @staticmethod
    def compute_indices(k, m):
        """
        Compute all orderings of m elements with values between [0, k-1]
        :param k: Number of alpha-vectors
        :param m: Number of observations
        :return: list of lists, where each list contains m elements, and each element is in [0, k-1].
        Total should be k^m elements
        """
        x = list(range(k))
        return [p for p in product(x, repeat=m)]

    def prune(self, n_states):
        """
        Remove dominated alpha-vectors using Lark's filtering algorithm
        :param n_states
        :return:
        """
        # parameters for linear program
        delta = 0.0000000001
        # equality constraints on the belief states
        A_eq = np.array([np.append(np.ones(n_states), [0.])])
        b_eq = np.array([1.])

        # dirty set
        F = self.gamma.copy()
        # clean set
        Q = set()

        for i in range(n_states):
            max_i = -np.inf
            best = None
            for av in F:
                if av.v[i] > max_i:
                    max_i = av.v[i]
                    best = av
            Q.update({best})
            F.remove(best)
        while F:
            av_i = F.pop()  # get a reference to av_i
            F.add(av_i)  # don't want to remove it yet from F
            dominated = False
            for av_j in Q:
                c = np.append(np.zeros(n_states), [1.])
                A_ub = np.array([np.append(-(av_i.v - av_j.v), [-1.])])
                b_ub = np.array([-delta])

                res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, None))
                if res.x[n_states] > 0.0:
                    # this one is dominated
                    dominated = True
                    F.remove(av_i)
                    break

            if not dominated:
                max_k = -np.inf
                best = None
                for av_k in F:
                    b = res.x[0:2]
                    v = np.dot(av_k.v, b)
                    if v > max_k:
                        max_k = v
                        best = av_k
                F.remove(best)
                if not self.check_duplicate(Q, best):
                    Q.update({best})
        self.gamma = Q

    @staticmethod
    def check_duplicate(a, av):
        """
        Check whether alpha vector av is already in set a

        :param a:
        :param av:
        :return:
        """
        for av_i in a:
            if np.allclose(av_i.v, av.v):
                return True
            if av_i.v[0] == av.v[0] and av_i.v[1] > av.v[1]:
                return True
            if av_i.v[1] == av.v[1] and av_i.v[0] > av.v[0]:
                return True

    @staticmethod
    def select_action(belief, vector_set):
        """
        Compute optimal action given a belief distribution
        :param belief: dim(belief) == dim(AlphaVector)
        :param vector_set
        :return:
        """
        max_v = -np.inf
        best = None
        for av in vector_set:
            v = np.dot(av.v, belief)

            if v > max_v:
                max_v = v
                best = av

        if best is None:
            raise ValueError('Vector set should not be empty')

        return best.action, best

In [28]:
states

('0', '1', '2', '3')

In [29]:
pomdp = POMDP(actions, t_prob, e_prob, rewards, states, discount)

In [30]:
pomdp_solve = ValueIteration(pomdp)

In [26]:
utility = pomdp_value_iteration(pomdp, epsilon=3)
utility

defaultdict(list,
            {'0': [array([ 219.62145794,   87.04320423,  -58.95578452, -185.0924472 ])]})

In [56]:
pd.DataFrame(np.array([204.52516018,   88.9820356 ,  -34.29024281, -171.26528113]))

,0
0,204.525160
1,88.982036
2,-34.290243
3,-171.265281


# ADQRN

In [29]:
from __future__ import division

import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

/home/yannik/miniconda3/envs/custom_hmm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yannik/miniconda3/envs/custom_hmm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yannik/miniconda3/envs/custom_hmm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yannik/miniconda3/envs/cu

In [30]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,13],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([13,2],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [31]:
def get_state(obs):
    state = 0
    diff = 16
    for i in range(len(statemean)):
        stateDiff = obs - statemean[i]
        stateDiffVal = np.sqrt(np.mean(stateDiff**2))
        if stateDiffVal < diff:
            diff = stateDiffVal
            state = i
    return state

In [32]:
# Getting the next step after an action is done

def getStepDetails(i,j,action):
    unitData = dataT_cycles[i]
    d = False
    if action == 1:
        newJ = 0
    else:
        newJ = j+1
    obsNext = unitData.values[newJ]
    if newJ >= len(unitData) - 1:
        d = True
    s1 = get_state(obsNext)
    r1 = rewards[action][s1]
    return r1,newJ,s1,obsNext,d

In [ ]:
# Set learning parameters
init = tf.global_variables_initializer()
y = discount
e = 0.1
num_episodes = len(dataT_cycles)
#create lists to contain total rewards and steps per episode
jList = []
rList = []
D = np.empty([0,5]) # Replay memory
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation for new unit
        rAll = 0
        d = False
        j = 0
        k = 0
        unitData = dataT_cycles[i]
        #The Q-Network
        while j < len(unitData):
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:unitData.values[j].reshape(1,13)})
            if np.random.rand(1) < e:
                a[0] = np.random.randint(0,2)
            #Get new state and reward from environment
            r,j,s1,o1,d = getStepDetails(i,j,a[0])
            D = np.vstack([D, [a[0],unitData.values[j-1].reshape(1,13),r,o1,s1]])
            if len(D) > 20:
                lastInd = np.random.randint(15,len(D))
                randomSample = D[lastInd-15:lastInd]
                finalO = D[lastInd,3].reshape(1,13)
                Reward = np.sum(D[lastInd-15:lastInd,2])
            else:
                finalO = o1.reshape(1,13)
                Reward = r
            # We take batch size of 15 (j in algorithm)
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:finalO})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = Reward + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:unitData.values[j-1].reshape(1,13),nextQ:targetQ})
            rAll += r
            s = s1
            k += 1
            if d == True or k >= 1000:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)

# Prediction

In [76]:
a = dataT_cycles[5].values[160].reshape(-1,13)

In [77]:
dataT_cycles[5].values[160]

array([0.77108434, 0.48484848, 0.40257649, 0.51515152, 0.03688414,
       0.61904762, 0.36886994, 0.55882353, 0.03333677, 0.65101962,
       0.58333333, 0.30232558, 0.2903894 ])

In [78]:
W1

array([[2364.409  , 5880.967  ],
       [1871.21   , 5603.3594 ],
       [3616.782  , 4988.657  ],
       [4809.125  ,  873.2698 ],
       [3126.0398 ,   76.49012],
       [3813.273  , 3731.947  ],
       [5509.8306 , 1830.7938 ],
       [4643.098  ,  543.59045],
       [8576.893  ,  -95.22542],
       [2755.4534 , 2314.9817 ],
       [2022.7578 , 2662.1064 ],
       [4247.6187 , 1322.4498 ],
       [2997.3079 , 1125.1122 ]], dtype=float32)

In [79]:
np.dot(a,W1)

array([[19181.12600722, 16785.20466186]])

### Validate that my modified HMM approach does well

In [23]:
startprob = np.array([1.0, 0.0, 0.0, 0.0])
# The transition matrix
transmat = np.array([[0.5, 0.5, 0.        , 0.        ],
       [0.        , 0.7, 0.3, 0.        ],
       [0.        , 0.        , 0.7, 0.3],
       [0.        , 0.        , 0.        , 1.        ]])
# The means of each component
means = np.array([[0.0],
                  [2.0],
                  [4.0],
                  [6.0]])
# The covariance of each component
covars = .5 * np.tile(np.identity(1), (4, 1, 1))

# Build an HMM instance and set parameters
model = hmm.GaussianHMM(n_components=4, covariance_type="full")

# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model.startprob_ = startprob
model.transmat_ = transmat
model.means_ = means
model.covars_ = covars

In [24]:
# Generate samples
X, Z = model.sample(1000)

In [25]:
X

array([[ 0.78741127],
       [-0.85468068],
       [ 1.92227986],
       [ 2.57110241],
       [ 2.34802956],
       [ 4.40175357],
       [ 6.47569351],
       [ 5.92391523],
       [ 5.61234941],
       [ 5.56644978],
       [ 5.87304739],
       [ 5.41978284],
       [ 5.81959748],
       [ 6.03645575],
       [ 6.41476863],
       [ 5.9034605 ],
       [ 6.3667654 ],
       [ 6.18113468],
       [ 5.13432466],
       [ 4.68990348],
       [ 5.43503211],
       [ 5.35102728],
       [ 5.72685409],
       [ 4.77774061],
       [ 6.58361492],
       [ 4.89415055],
       [ 6.24063154],
       [ 5.80958982],
       [ 5.1522272 ],
       [ 6.66697103],
       [ 6.1722104 ],
       [ 4.83741061],
       [ 5.93581895],
       [ 5.97718146],
       [ 6.20925034],
       [ 5.96357562],
       [ 7.08130902],
       [ 6.01362725],
       [ 4.9751615 ],
       [ 5.22894194],
       [ 5.78982243],
       [ 5.68638644],
       [ 6.10442263],
       [ 6.715448  ],
       [ 5.96245544],
       [ 6

In [26]:
Z

array([0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [27]:
lr = hmm.GaussianHMM(n_components=4, covariance_type="diag",init_params="cm", params="mtc")
lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]])
lr.fit(X)

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='cm', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=4, n_iter=10, params='mtc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

In [28]:
lr.transmat_

array([[0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.66666664, 0.33333336],
       [0.        , 0.        , 0.        , 1.        ]])

In [29]:
pd.DataFrame(lr.transmat_)

,0,1,2,3
0,0.0,1.0,0.000000,0.000000
1,0.0,0.0,1.000000,0.000000
2,0.0,0.0,0.666667,0.333333
3,0.0,0.0,0.000000,1.000000


In [30]:
lr.means_, lr.covars_

(array([[ 0.78741127],
        [-0.85468068],
        [ 2.2804706 ],
        [ 6.01303355]]), array([[[0.01      ]],
 
        [[0.01      ]],
 
        [[0.07577723]],
 
        [[0.45954586]]]))

In [31]:
lr.predict(X)

array([0, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,